In [7]:
import os

from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

load_dotenv()

model = "openai/gpt-5-mini"
temperature = 0.0

llm = ChatOpenAI(
    model=model,
    api_key=os.getenv("OPENROUTER_API_KEY"),
    base_url="https://openrouter.ai/api/v1",
    temperature=temperature,
    reasoning_effort="medium",
    include_response_headers=True,
)

In [8]:
response = llm.invoke("Explain gradient descent")

2025-11-25 17:16:47,159 - INFO - HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"


In [9]:
response

AIMessage(content='Short definition\n- Gradient descent is an iterative optimization algorithm that finds (local) minima of a differentiable function by moving parameters in the direction opposite to the gradient.\n\nIntuition\n- The gradient at a point points in the direction of steepest increase. Moving a small step opposite the gradient decreases the function value most quickly locally. Repeating these steps moves you toward a minimum.\n\nMathematics (core update)\n- Given a loss/objective J(θ) and learning rate η > 0:\n  θ ← θ − η ∇J(θ)\n- ∇J(θ) is the vector of partial derivatives (the gradient). The negative sign makes the update go downhill.\n\nSimple 1D example\n- Let f(x) = (x − 3)^2. Then f′(x) = 2(x − 3).\n- Update: x ← x − η·2(x − 3).\n- If x0 = 0 and η = 0.1, x1 = 0 + 0.6 = 0.6. Repeating moves x toward 3.\n\nVariants\n- Batch gradient descent: use the full dataset to compute ∇J (stable but expensive for large data).\n- Stochastic gradient descent (SGD): use the gradient f

In [10]:
from langchain_playground.llm.openrouter import parse_invoke

parse_invoke(response, include_reasoning=True)

(None,
 'Short definition\n- Gradient descent is an iterative optimization algorithm that finds (local) minima of a differentiable function by moving parameters in the direction opposite to the gradient.\n\nIntuition\n- The gradient at a point points in the direction of steepest increase. Moving a small step opposite the gradient decreases the function value most quickly locally. Repeating these steps moves you toward a minimum.\n\nMathematics (core update)\n- Given a loss/objective J(θ) and learning rate η > 0:\n  θ ← θ − η ∇J(θ)\n- ∇J(θ) is the vector of partial derivatives (the gradient). The negative sign makes the update go downhill.\n\nSimple 1D example\n- Let f(x) = (x − 3)^2. Then f′(x) = 2(x − 3).\n- Update: x ← x − η·2(x − 3).\n- If x0 = 0 and η = 0.1, x1 = 0 + 0.6 = 0.6. Repeating moves x toward 3.\n\nVariants\n- Batch gradient descent: use the full dataset to compute ∇J (stable but expensive for large data).\n- Stochastic gradient descent (SGD): use the gradient from one tr

In [1]:
"""OpenRouter LLM client initialization and configuration."""

import os

from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

load_dotenv()

model = "openai/gpt-5-mini"
temperature = 0.0


class Schema(BaseModel):
    answer: str = Field(description="The answer to the question")
    reason: str = Field(description="The reason for the answer")


llm = ChatOpenAI(
    model=model,
    api_key=os.getenv("OPENROUTER_API_KEY"),
    base_url="https://openrouter.ai/api/v1",
    temperature=temperature,
    # reasoning_effort="medium",
    reasoning={
        "effort": "medium",  # Can be "low", "medium", or "high"
        "summary": "auto",  # Can be "auto", "concise", or "detailed"
    },
).with_structured_output(Schema)

In [3]:
from langchain_playground.llm.multimodal import MediaMessage

message = MediaMessage("input.png", description="Describe the weather in this image")
ai_msg = llm.invoke([message])
ai_msg

2025-11-25 17:27:11,591 - INFO - HTTP Request: POST https://openrouter.ai/api/v1/responses "HTTP/1.1 200 OK"


Schema(answer='I can’t tell the weather from this image. It’s a screenshot of code (a message dictionary showing a text and image entry), not a photo of an outdoor scene.', reason='The image contains a code snippet and no visual cues (sky, clouds, sun, precipitation, etc.) that would indicate weather. If you upload or link to a photo of the environment, I can describe the weather from that image.')

In [ ]:
# With structured output, response is already a Pydantic model
response: Schema = llm.invoke("What is the price of GOOGL today 25-Nov-2025")
print(f"Answer: {response.answer}")
print(f"Reason: {response.reason}")
response

Answer: As of November 25, 2025, GOOGL (Alphabet Inc.) is trading at $318.58 USD per share. Latest trade time in the feed: 2025-11-25 01:15:00 UTC. Intraday high / low: $327.18 / $305.06. Today's open: $310.81. Change vs previous close: +$19.01 (≈+6.34%). Intraday volume: 85,165,123. Market cap: ~$2.94T. citeturn0finance0
Reason: I fetched the live market quote for GOOGL from a financial data feed (finance lookup) and reported the price and key intraday stats along with the feed's timestamp. If you want the price in a specific U.S. timezone, a live chart, or an updated quote, I can fetch that now.


Schema(answer="As of November 25, 2025, GOOGL (Alphabet Inc.) is trading at $318.58 USD per share. Latest trade time in the feed: 2025-11-25 01:15:00 UTC. Intraday high / low: $327.18 / $305.06. Today's open: $310.81. Change vs previous close: +$19.01 (≈+6.34%). Intraday volume: 85,165,123. Market cap: ~$2.94T. \ue200cite\ue202turn0finance0\ue201", reason="I fetched the live market quote for GOOGL from a financial data feed (finance lookup) and reported the price and key intraday stats along with the feed's timestamp. If you want the price in a specific U.S. timezone, a live chart, or an updated quote, I can fetch that now.")

In [ ]:
from langchain.tools import tool

from langchain_playground.tools.youtube import youtube_loader


@tool
def scrape_youtube_video(youtube_url: str) -> str:
    """Scrape a YouTube video and return its transcript and metadata.

    This tool extracts the transcript, title, channel, duration, views, likes,
    and other metadata from a YouTube video URL.

    Args:
        youtube_url: The YouTube video URL to scrape (e.g., https://www.youtube.com/watch?v=VIDEO_ID)

    Returns:
        A formatted string containing video metadata and transcript
    """
    return youtube_loader(youtube_url)


tools = [scrape_youtube_video]

2025-11-25 16:59:32,576 - WARNING - USER_AGENT environment variable not set, consider setting it to identify your requests.


In [ ]:
from langchain.agents import create_agent

# Create a model for the agent (without structured output, as agents handle tool calling)
agent_llm = ChatOpenAI(
    model=model,
    api_key=os.getenv("OPENROUTER_API_KEY"),
    base_url="https://openrouter.ai/api/v1",
    temperature=temperature,
    reasoning_effort="medium",
    extra_body={"plugins": [{"id": "web"}]},
)

# Create a simple agent with the YouTube scraper tool
youtube_agent = create_agent(
    model=agent_llm,
    tools=tools,
    system_prompt="You are a helpful assistant that can analyze YouTube videos. When given a YouTube URL, use the scrape_youtube_video tool to get the video transcript and metadata, then provide insights or answer questions about the video content.",
)

In [ ]:
# Example: Scrape a YouTube video and ask questions about it
# Replace with your own YouTube URL
youtube_url = "https://www.youtube.com/watch?v=dQw4w9WgXcQ"  # Example URL

response = youtube_agent.invoke({"messages": [{"role": "user", "content": f"Please scrape this YouTube video: {youtube_url} and summarize the main points discussed in the transcript."}]})

print(response["messages"][-1].content)

2025-11-25 16:59:34,697 - INFO - HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
2025-11-25 16:59:42,210 - INFO - HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"


I scraped the video and its transcript. Short summary first, then the main points.

Video metadata (from the scrape)
- Title: Rick Astley - Never Gonna Give You Up (Official Video) (4K Remaster)
- Channel: Rick Astley
- Duration: 3:34
- Published: Oct 24, 2009
- Views/likes (at scrape): ~1.72B views, ~18.6M likes

Summary
- The transcript is the lyrics to the song “Never Gonna Give You Up.” The central message is a repeated, emphatic promise of loyalty, commitment, and reassurance from the singer to a romantic partner.

Main points/themes in the lyrics
- Promise of unwavering commitment: “A full commitment’s what I’m thinking of,” and repeated lines “Never gonna give you up / Never gonna let you down.”
- Reassurance against hurt or abandonment: vows not to run around, desert, make cry, say goodbye, tell lies or hurt the partner.
- Shared history and mutual understanding: “We’ve known each other for so long,” implying familiarity and emotional closeness.
- Acknowledgement of emotional p